In [ ]:
#Run cell to mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# install package to have access to custom functions
%pip install /content/drive/MyDrive/EMG_gestures/ --use-feature=in-tree-build

Processing ./drive/MyDrive/EMG_gestures
  Created wheel for EMG-gestures: filename=EMG_gestures-0.1.0-py3-none-any.whl size=45275 sha256=0c919130b37a99e0e31f96bda5694972597dc9b384d819a1cd2709c2562fba48
  Stored in directory: /tmp/pip-ephem-wheel-cache-cw28x0nv/wheels/a2/b7/61/2147fa082a9e51bef5dcc38dd3f0898fe0554d62203c0e383e
Successfully built EMG-gestures
  Attempting uninstall: EMG-gestures
    Found existing installation: EMG-gestures 0.1.0
    Uninstalling EMG-gestures-0.1.0:
      Successfully uninstalled EMG-gestures-0.1.0


In [ ]:
#import necessary packages

#our workhorses
import numpy as np
import pandas as pd
import scipy

#to visualize
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#style params for figures
sns.set(font_scale = 2)
plt.style.use('seaborn-white')
plt.rc("axes", labelweight="bold")
from IPython.display import display, HTML

#to load files
import os
import sys
import h5py

#import cusotm functions
from EMG_gestures.utils import *
from EMG_gestures.analysis import across_subject_nn_performance



In [ ]:
#define where the data files are located
data_folder = '/content/drive/MyDrive/EMG_gestures/EMG_data/'
results_folder = '/content/drive/MyDrive/EMG_gestures/results_data/single_subject_training/simple_NN/'

nsubjects = 36


# User-defined parameters
lo_freq = 20 #lower bound of bandpass filter
hi_freq = 450 #upper bound of bandpass filter

win_size = 100 #define window size over which to compute time-domain features
step = win_size #keeping this parameter in case we want to re-run later with some overlap


#for NN training
epochs = 1000
batch_size = 2
es_patience = 5
model_dict = {'fe_layers':1, 'fe_activation':'tanh'}


nreps = 10
exclude = [0,7]#labels to exclude
#performance metrics
score_list = ['f1','accuracy']


for src_subject_id in range(2,nsubjects+1):

    results_df = across_subject_nn_performance(data_folder, src_subject_id, nsubjects, nreps, lo_freq, hi_freq, win_size, step, model_dict,\
                                            exclude, score_list, epochs = epochs, batch_size = batch_size, mv = None, permute = False)

    results_df['Shuffled'] = False

    perm_results_df = across_subject_nn_performance(data_folder, src_subject_id, nsubjects, nreps, lo_freq, hi_freq, win_size, step, model_dict,\
                                                    exclude, score_list, epochs = epochs, batch_size = batch_size, mv =None, permute = True)

    perm_results_df['Shuffled'] = True

    #concatenate

    results_df = pd.concat([results_df, perm_results_df]).reset_index(drop = True)
    results_df['Train_Subject'] = src_subject_id

    # #save results to file
    results_fn = 'subject_%02d_across_subject_results.h5'%(src_subject_id)
    results_df.to_hdf(os.path.join(results_folder,results_fn), key='results_df', mode='w')

Streaming output truncated to the last 5000 lines.
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/21
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/22
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/23
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/24
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/25
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/26
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/27
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/28
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/29
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/30
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/31
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/32
Evaluate Model
T

In [ ]:
results_df

,Type,Rep,Test_Subject,Epochs,Batch_Size,Train_Loss,Val_Loss,Epochs_Trained,f1_score,accuracy_score,Shuffled,Train_Subject
0,Test,1,2,1000,2,0.261521,0.388965,50.0,0.275453,0.313397,False,1
1,Test,1,3,1000,2,0.261521,0.388965,50.0,0.735143,0.731579,False,1
2,Test,1,4,1000,2,0.261521,0.388965,50.0,0.624309,0.661137,False,1
3,Test,1,5,1000,2,0.261521,0.388965,50.0,0.679905,0.712468,False,1
4,Test,1,6,1000,2,0.261521,0.388965,50.0,0.549720,0.614796,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...
715,Train,6,1,1000,2,1.582886,2.148539,6.0,0.346116,0.380841,True,1
716,Train,7,1,1000,2,1.259061,2.551494,8.0,0.367510,0.408879,True,1
717,Train,8,1,1000,2,1.511088,2.134221,6.0,0.310795,0.369159,True,1
718,Train,9,1,1000,2,1.554581,2.190107,6.0,0.314464,0.352804,True,1


In [ ]:
results_df.groupby(['Type','Shuffled']).mean()

Rep  Test_Subject  ...  accuracy_score  Train_Subject
Type  Shuffled                     ...                               
Test  False     5.5          19.0  ...        0.604131            1.0
      True      5.5          19.0  ...        0.191862            1.0
Train False     5.5           1.0  ...        0.908879            1.0
      True      5.5           1.0  ...        0.396028            1.0

[4 rows x 10 columns]